<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_02/7%20-%20C2_W2_Lab_1_cats_v_dogs_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Aumento de dados

Nas lições anteriores, você viu que ter uma alta acurácia de treinamento não significa automaticamente ter um bom modelo preditivo.

Ele ainda pode ter um desempenho ruim em novos dados porque se ajustou demais ao conjunto de treinamento.

Neste laboratório, você verá como evitar isso usando _data augmentation_.

Isso aumenta a quantidade de dados de treinamento modificando as propriedades dos dados de treinamento existentes. 

Por exemplo, em dados de imagem, você pode aplicar diferentes técnicas de pré-processamento, como: girar, inverter, cortar ou aplicar zoom nas imagens existentes para simular outros dados com os quais o modelo também deve aprender. 

Dessa forma, o modelo veria mais variedade nas imagens durante o treinamento e, assim, inferiria melhor sobre dados novos e não vistos anteriormente.

Vamos ver como isso pode ser feito nas seções a seguir.

## Desempenho da linha de base

Você começará com um modelo que é muito eficaz no aprendizado de `Cats vs Dogs` sem aumento de dados.

Ele é semelhante aos modelos anteriores que você usou.

Observe que há quatro camadas convolucionais com 32, 64, 128 e 128 convoluções, respectivamente.

O código é basicamente o mesmo do laboratório anterior, portanto, não analisaremos os detalhes passo a passo, pois você já o viu antes.

Você treinará apenas 20 épocas para economizar tempo, mas fique à vontade para aumentar esse número se quiser.

In [ ]:
# Baixar o conjunto de dados
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

# Descompactar o arquivo
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Atribuir diretórios de conjuntos de treinamento e validação
base_dir = 'tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Diretório com fotos de treinamento de gatos
train_cats_dir = os.path.join(train_dir, 'cats')

# Diretório com fotos de treinamento de cães
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Diretório com imagens de gatos para validação
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Diretório com fotos de cães para validação
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Você colocará a criação do modelo dentro de uma função para poder inicializar facilmente um novo modelo quando usar a aumento de dados mais adiante neste notebook.

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_model():
  '''Cria uma CNN com 4 camadas convolucionais'''
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])
  
  return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas as imagens serão redimensionadas em 1,/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Fluxo de imagens de treinamento em lotes de 20 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        train_dir, # Esse é o diretório de origem das imagens de treinamento
        target_size=(150, 150),  # Todas as imagens serão redimensionadas para 150x150
        batch_size=20,
        # Como usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')

# Imagens de validação de fluxo em lotes de 20 usando o gerador test_datagen
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

In [ ]:
# Constante para épocas
EPOCHS = 20

# Criar um novo modelo
model = create_model()

# Treinar o modelo
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 imagens = batch_size * steps
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 imagens = batch_size * steps
      verbose=2)

Em seguida, você visualizará a perda e a acurácia em relação ao conjunto de treinamento e validação.

Você usará novamente uma função auxiliar para que ela possa ser reutilizada posteriormente.

Essa função aceita um objeto [History](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History) que contém os resultados do método `fit()` que você executou acima.

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_acc(history):
  '''Plota a perda e a Acurácia do treinamento e da validação de um objeto de histórico'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Acurácia do treinamento')
  plt.plot(epochs, val_acc, 'b', label='Acurácia da validação')
  plt.title('Acurácia do treinamento e da validação')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Perda de Treino')
  plt.plot(epochs, val_loss, 'b', label='Perda de Validação')
  plt.title('Perda de treinamento e validação')
  plt.legend()

  plt.show()

In [ ]:
# Plotar os resultados do treinamento
plot_loss_acc(history)

Nos resultados acima, você verá que a acurácia do treinamento é superior a 90% e a acurácia da validação está na faixa de 70% a 80%.

Esse é um ótimo exemplo de _overfitting_, o que, em resumo, significa que ele pode se sair muito bem com imagens que já viu antes, mas não tão bem com imagens que não viu.

## Aumento de dados

Um método simples para evitar o _overfitting_ é alterar um pouco as imagens.

Se você pensar bem, a maioria das imagens de um gato é muito semelhante: as orelhas estão no topo, depois os olhos, depois a boca etc.

Coisas como a distância entre os olhos e as orelhas também serão sempre muito semelhantes. 

E se você ajustar um pouco as imagens: girar a imagem, comprimi-la, etc.?  É disso que se trata o aumento de imagens. E há uma API que facilita isso!

Dê uma olhada no [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) que você tem usado para redimensionar a imagem. Há outras propriedades nele que você pode usar para aumentar a imagem. 

```
# Atualizado para fazer o aumento da imagem
train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
```

Essas são apenas algumas das opções disponíveis. Vamos examiná-las rapidamente:

* `rotation_range` é um valor em graus (0-180) dentro do qual as imagens serão giradas aleatoriamente.
* `width_shift` e `height_shift` são intervalos (como uma fração da largura ou altura total) dentro dos quais as imagens podem ser convertidas aleatoriamente na vertical ou na horizontal.
* O `shear_range` serve para aplicar transformações de cisalhamento aleatoriamente.
* O `zoom_range` serve para aplicar zoom aleatoriamente nas imagens.
* `horizontal_flip` serve para inverter metade das imagens horizontalmente de forma aleatória. Isso é relevante quando não há suposições de assimetria horizontal (por exemplo, imagens do mundo real).
* O `fill_mode` é a estratégia usada para preencher os pixels recém-criados, que podem aparecer após uma rotação ou uma mudança de largura/altura.


Execute as próximas células para ver o impacto nos resultados. O código é semelhante ao da linha de base, mas a definição de `train_datagen` foi atualizada para usar os parâmetros descritos acima.


In [ ]:
# Criar um novo modelo
model_for_aug = create_model()

# Esse código foi alterado. Agora, em vez de o ImageGenerator apenas redimensionar
# a imagem, também giramos e fazemos outras operações
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Fluxo de imagens de treinamento em lotes de 20 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        train_dir,  # Esse é o diretório de origem das imagens de treinamento
        target_size=(150, 150),  # Todas as imagens serão redimensionadas para 150x150
        batch_size=20,
        # Como usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')

# Fluxo de Imagens de validação em lotes de 20 usando o gerador test_datagen
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# Treinar o novo modelo
history_with_aug = model_for_aug.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 imagens = batch_size * steps
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 imagens = batch_size * steps
      verbose=2)

In [ ]:
# Plote os resultados do treinamento com aumento de dados
plot_loss_acc(history_with_aug)

Como você pode ver, a acurácia do treinamento diminuiu em comparação com a linha de base.

Isso é esperado porque (como resultado do aumento de dados) há mais variedade nas imagens e, portanto, o modelo precisará de mais execuções para aprender com elas.

O ponto positivo é que a acurácia da validação não está mais estagnada e está mais alinhada com os resultados do treinamento.

Isso significa que o modelo agora está apresentando melhor desempenho em dados não vistos.

## Encerramento

Este exercício mostrou um truque simples para evitar o _overfitting_.

Você pode melhorar seus resultados de linha de base simplesmente ajustando as mesmas imagens que já possui.

A classe `ImageDataGenerator` tem parâmetros incorporados para fazer exatamente isso.

Tente modificar um pouco mais os valores no `train_datagen` e veja os resultados que você obtém.

Observe que isso não funcionará em todos os casos.

Na próxima atividade, você verá um cenário em que o aumento de dados não ajudará a melhorar a acurácia da validação.